## 1. 라이브러리 가져오기

In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'NanumGothic'
warnings.filterwarnings(action='ignore')

C:\Users\bunga\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### 데이터 가져오기

In [2]:
retail = pd.read_excel('data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx', header=1)
raw_retail = pd.read_excel('data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx', header=1) #원본데이터 보존

#### 데이터 확인하기

In [3]:
retail.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


In [4]:
retail.info() #노출(분), 취급액 항목 na값 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    38309 non-null  datetime64[ns]
 1   노출(분)   21525 non-null  float64       
 2   마더코드    38309 non-null  int64         
 3   상품코드    38309 non-null  int64         
 4   상품명     38309 non-null  object        
 5   상품군     38309 non-null  object        
 6   판매단가    38309 non-null  int64         
 7   취급액     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [5]:
retail.describe() #min 음수는 없다.

,노출(분),마더코드,상품코드,판매단가,취급액
count,21525.000000,38309.000000,38309.000000,3.830900e+04,3.737200e+04
mean,20.174664,100390.972148,201219.923334,4.565537e+05,2.187305e+07
std,3.633757,249.927870,735.677344,7.260555e+05,2.019140e+07
min,2.466667,100000.000000,200000.000000,0.000000e+00,5.000000e+04
25%,20.000000,100155.000000,200550.000000,5.900000e+04,6.880750e+06
50%,20.000000,100346.000000,201167.000000,1.090000e+05,1.612950e+07
75%,20.000000,100596.000000,201863.000000,4.990000e+05,3.163125e+07
max,60.000000,100849.000000,202513.000000,7.930000e+06,3.220090e+08


In [6]:
retail.isnull().sum() #방송일시, 취급액 NA 값 확인

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
dtype: int64

#### 데이터 추출 조건 

In [7]:
#중요! 같은 시간대 방송 중 하나만 추출해주는 조건
notNA = raw_retail['노출(분)'].notna()

-----------

## 2. 데이터 전처리 (Data Cleansing & Pre-Processing)

#### 1) 변수 추가
1. 판매량 (취급액 / 판매단가)
2. 방송일시 시간 세분화(일자, 월, 일, 시간, 시, 분)
3. 계절 (3개월 단위)

In [8]:
#판매량
retail['판매량'] = retail['취급액'] / retail['판매단가']

#시간
retail['date'] = retail['방송일시'].dt.date         # YYYY-MM-DD(문자)
retail['month'] = retail['방송일시'].dt.month        # 월(숫자)
retail['day'] = retail['방송일시'].dt.day          # 일(숫자)
retail['time'] = retail['방송일시'].dt.time         # HH:MM:SS(문자)
retail['hour'] = retail['방송일시'].dt.hour         # 시(숫자)
retail['minute'] = retail['방송일시'].dt.minute       # 분(숫자)

#계절
retail['계절'] = 0
retail['계절'][retail['방송일시'] < pd.to_datetime('20200301')] = '겨울'
retail['계절'][retail['방송일시'] < pd.to_datetime('20191201')] = '가을'
retail['계절'][retail['방송일시'] < pd.to_datetime('20190901')] = '여름'
retail['계절'][retail['방송일시'] < pd.to_datetime('20190601')] = '봄'
retail['계절'][retail['방송일시'] < pd.to_datetime('20190301')] = '겨울'

#### 2) 변수명 변경 (미정)

#### 3) 노출시간 전처리
- 반올림
- imputation
- 그룹화(20미만, 20, 20~30, 30, 30 초과)

In [9]:
retail['노출(분)'] = retail['노출(분)'].round() #반올림
retail['노출(분)'].fillna(method='ffill', inplace=True) #Imputation
retail['노출(분)'].isnull().sum()

0

#### 4) Business 모델에 안 맞는 데이터 제거
- "취급액 < 판매단가"인 경우, 취급액을 0원으로 일괄적으로 처리

In [10]:
raw_retail[raw_retail['판매단가'] > raw_retail['취급액']]['취급액'].value_counts()

50000.0    1990
Name: 취급액, dtype: int64

In [11]:
retail['취급액'][retail['판매단가'] > retail['취급액']] = 0

In [14]:
#retail.to_csv('data/retail.csv')

---

## 3. 탐색적 자료분석 (EDA)
### 상품군별 분석
- 손지우
 1. 생활용품: household
 2. 가구: furniture
 3. 침구: bedding
 4. 무형: etc

In [ ]:
#상품군별 실제 방송횟수
a = retail[raw_retail['노출(분)'].notna()].groupby('상품군').size() #.sort_values(ascending=False)
plt.xticks(rotation=45)
for x, y in zip(a.index, a):
    plt.text(x, y, s=y)
plt.title('상품군별 방송 횟수')
plt.ylabel('방송 횟수')
plt.plot(a, c='orange')
plt.show()

In [ ]:
#상품군별 취급액 최대값
#수정필요...같은 방송시간대는 취급액 합쳐야함
a = retail[retail['상품군']!='무형'].groupby('상품군')['취급액'].max() #무형은 0이라서 제외
plt.xticks(rotation=45)
for x, y in zip(a.index, a):
    plt.text(x, y, s=y/10000)
plt.title('상품군별 취급액 최대값')
plt.ylabel('취급액(만원)')
plt.plot(a, c='orange')
plt.show()

In [ ]:
#상품군별 판매량 최대값
#수정필요...같은 방송시간대는 판매량 합쳐야함
a = retail[retail['상품군']!='무형'].groupby('상품군')['판매량'].max() #무형은 0이라서 제외
plt.xticks(rotation=45)
for x, y in zip(a.index, a):
    plt.text(x, y, s=int(y))
plt.title('상품군별 판매량 최대값')
plt.ylabel('판매량(개)')
plt.plot(a, c='orange')
plt.show()

In [ ]:
#상품군별 판매량 평균값
#수정필요...같은 방송시간대는 판매량 합쳐야함
a = retail[retail['상품군']!='무형'].groupby('상품군')['판매량'].mean() #무형은 0이라서 제외
plt.xticks(rotation=45)
for x, y in zip(a.index, a):
    plt.text(x, y, s=int(y))
plt.title('상품군별 판매량 평균값')
plt.ylabel('판매량(개)')
plt.plot(a, c='orange')
plt.show()

결론: **상품군별** 정보가 상이하므로, 상품군에 따라 나눠서 EDA를 접근하는 것이 좋겠다.

In [ ]:
#상품군별 접근
household = retail.groupby('상품군').get_group('생활용품')
furniture = retail.groupby('상품군').get_group('가구')
bedding = retail.groupby('상품군').get_group('침구')
etc = retail.groupby('상품군').get_group('무형')

## 1) 생활용품

<img src="https://image.freepik.com/free-vector/household-goods-shop-icon-set_81894-628.jpg" width="200" height="200">

#### 1) 방송일시

In [ ]:
pass

#### 2) 노출(분)

In [ ]:
household[notNA].groupby('노출(분)').size()
#대부분 20분 편성이며, 다음으로는 30분 편성이 많다.
#나머지들을 구간으로 나눠서 그룹화하는게 좋을 듯 하다.

In [ ]:
household['노출(분)'][household['노출(분)'] < 20] = 10 #임의로 10 배정(실제평균은 13)
household['노출(분)'][(household['노출(분)'] > 20) & (household['노출(분)'] < 30)] = 25 #임의로 25 배정
household.groupby('노출(분)').size()

In [ ]:
fig, ax = plt.subplots()
a = household[notNA].groupby('노출(분)').size() #notNA 적용
labels = ['20분 미만','20분','20분이상 30분 미만','30분']
plt.bar(labels, a, color='orange')
plt.title('생활용품 노출(분)')
plt.xlabel('시간(분)')
plt.ylabel('개수')
for x, y in zip(labels, a):
     plt.text(x, y+5, s='{}회'.format(y))
plt.show()

#### 3) 마더코드

In [ ]:
household.groupby('마더코드').size().sort_values(ascending=False)
# 76개의 브랜드가 있는 것으로 추정된다.

In [ ]:
#가장 많은 마더코드 top 5
top5 = household.groupby('마더코드').size().sort_values(ascending=False)[:5].reset_index()['마더코드']
top5

#### 4) 상품코드

In [ ]:
pass

#### 5) 상품명

#### 상품명 전처리 아이디어
1. 브랜드
 - 특수문자: 공백으로 만들기
 - 상품명이 먼저 등장하는 경우: ex) KF94, 대형, ...
2. 세트 or Not
 - 동시에 판 것들
3. 노출빈도

In [ ]:
#텍스트에 포함되어 있는 특수 문자를 공백으로 바꾸기
def cleanText(readData):
    text = re.sub('[-=,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', readData) # '+'는 그대로 놔두기
    return text

In [ ]:
#test
cleanText('가)세렌셉템버 다이아 체크 리빙박스+압축팩') #test /'+'는 그대로 놔두기

#### 5-1) 브랜드

In [ ]:
#브랜드 띄어쓰기 교정
def makeSpaceBrand(brand, data):
    new_brand = []
    for b in brand:
        b_with_space = ' ' + b + ' '
        tmp = data.replace(b, b_with_space)
        
        if b in tmp.split():
            new_brand.append(b)
            break
        else:
            new_brand.append(data)
            break          
    return new_brand

In [ ]:
#test
print(makeSpaceBrand(['까사마루', '올바로', '보국'],'가까사마루행거 2단'))
print(makeSpaceBrand(['까사마루', '올바로', '보국'],'까사마행거 2단'))

In [ ]:
household_brand = []
exception = ['가','무이자', '일시불', '국내산', '김병만의', '김병지', '완벽더블구성', '기본구성', '파격가', '일','무',
            '2019년', 'D', 'ALL', 'New', '1세트','2세트','5세트', '국내제작', '중형','점보특대형','점보형',
            '퀸+퀸','킹+싱글','퀸+싱글','킹사이즈','퀸사이즈','싱글사이즈', '더블+더블','더블+싱글','더블사이즈','싱글사이즈',
            '초특가', '1+1', '풀패키지','실속패키지', '국내제조','한세트','붙이는', 'KF94', '12', '싱글+싱글']
brand_to_check = ['까사마루', '올바로', '엔웰스']

#3D매쉬 -> '두씽'으로 브랜드 바꿔줘야함
#파워스윙 -> '스윙'으로 브랜드 바꿔줘야함
#1+1보국미니히터(화이트+레드)BKH-1083P +BKH-1083PR -> '보국'을 꺼내야함
#OK 근육통완화 동전패치 -> '케이원헬스케어'
#

for line in household['상품명']:
    line = makeSpaceBrand(brand_to_check, line) #브랜드 띄어쓰기 교정
    line = cleanText(str(line)) #str 변환 후, 특수문자 제거
    tmp = line.split()
    if tmp[0] not in exception:
        household_brand.append(tmp[0])
    elif tmp[1] not in exception:
        household_brand.append(tmp[1])
    elif tmp[2] not in exception:
        household_brand.append(tmp[2])
    else:
        household_brand.append(tmp[3])

print(set(household_brand))
print('\n', len(set(household_brand)), "개의 생활용품 브랜드가 있다.")

#### 6) 판매단가

In [ ]:
household['판매단가'].sort_values()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,8), sharey=True)
sns.distplot(household['판매단가'][household['판매단가'] < 100000], ax=axes[0])
sns.distplot(household['판매단가'][household['판매단가'] >= 100000], ax=axes[1])
plt.show()

#### 7) 취급액

In [ ]:
household['취급액'].sort_values()

In [ ]:
household['취급액'][household['취급액']==0].count()

In [ ]:
fig, axes = plt.subplots(figsize=(15,8))
sns.distplot(household['취급액'][household['취급액']!=0])
plt.show()

#### 8) 판매량

In [ ]:
household['판매량'].sort_values()

In [ ]:
household['판매량'][household['판매량']==0].count()

In [ ]:
fig, axes = plt.subplots(figsize=(15,8))
sns.distplot(household['판매량'][household['판매량']!=0])
plt.show()

#### 9) 계절

In [ ]:
household[raw_retail['노출(분)'].notna()].groupby('계절').size()
#생활용품은 여름에는 적게 방송했고, 가을에 가장 많이 방송했다.

In [ ]:
a = household[raw_retail['노출(분)'].notna()].groupby('계절').size()
a = a.reindex(index = ['봄','여름','가을','겨울'])

fix, ax = plt.subplots()
for x, y in zip(a.index, a):
    plt.text(x, y, '{}회'.format(y))
plt.plot(a, c='orange')
plt.show()

#### 10) 시간

In [ ]:
#월별
a = household[raw_retail['노출(분)'].notna()].groupby('month').size()

fig, ax = plt.subplots()
plt.title('생활용품 월별 방송횟수')
plt.xticks(range(1,13), [str(month) + '월' for month in list(range(1,13))])
for x, y in zip(range(1, 13), a):
    plt.text(x, y-5, "{}회".format(y))
plt.plot(a, c='orange')
plt.show()

## 종합

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
a = sns.heatmap(household[['노출(분)','판매단가','취급액','판매량']].corr(), annot=True, linewidths=1, cmap='RdYlGn_r')
ax.set_title('생활용품 상관관계분석')
plt.show()

--------